In [1]:
import tensorflow as tf
import datetime

In [2]:
tf.__version__

'2.0.0'

In [3]:
(train_image, train_labels),(test_image, test_labels) = tf.keras.datasets.mnist.load_data()  #mnist数据集中第二个是test数据集，用不着所以使用占位符

In [4]:
train_image.shape

(60000, 28, 28)

In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [6]:
train_image.shape

(60000, 28, 28)

In [7]:
train_image = tf.expand_dims(train_image,-1)#扩展维度其他方法:1. tf.newx 2. napy里的reshape
test_image = tf.expand_dims(test_image,-1)

In [8]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [9]:
train_image = tf.cast(train_image/255, tf.float32)#归一化，转换数据类型
test_image = tf.cast(test_image/255, tf.float32)#归一化，转换数据类型

In [10]:
train_labels = tf.cast(train_labels, tf.int64)#转换数据类型
test_labels = tf.cast(test_labels, tf.int64)

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_labels))#扩展维度

In [12]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_labels))#扩展维度

In [13]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [14]:
dataset = dataset.shuffle(10000).batch(32)
test_dataset = test_dataset.batch(32)

In [15]:
dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [16]:
#7.5 第五节

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3],activation='relu',input_shape=(None,None,1)),
    tf.keras.layers.Conv2D(32,[3,3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),#全局平均池化
    tf.keras.layers.Dense(10,activation='softmax')#这里不需要softmax的激活方式，softmax只是把值归一化
])

In [18]:
#因为我们要自定义循环，不需要编译模型

In [19]:
#定义优化函数
optimizer = tf.keras.optimizers.Adam()

In [20]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy() #大写的是可调用的方法 模型没有激活，所以传递参数from_logits=True

In [21]:
def loss(model, x, y):
    y_ = model(x) #y是实际的label y_是预测的label
    return loss_func(y, y_)    #建立损失函数

In [22]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy') #初始化计算对象

In [23]:
test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy') #初始化计算对象

In [24]:
def train_step(model,images, labels):
    #计算梯度 计算损失函数与可model的变量训练参数的梯度
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels, pred)
    grads = t.gradient(loss_step, model.trainable_variables)  # 计算损失函数与可model的变量训练参数的梯度
    optimizer.apply_gradients(zip(grads, model.trainable_variables)) #优化 改变可训练参数变量的值
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [25]:
def test_step(model, images, labels):
        pred = model(images)
        loss_step = loss_func(labels, pred)
        test_loss(loss_step)
        test_accuracy(labels, pred)

In [26]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #名称为时间的目录

In [27]:
train_log_dir = 'logs/gradient_tape' + current_time + '/train'
test_log_dir = 'logs/gradient_tape' + current_time + '/test'
train_writer = tf.summary.create_file_writer(train_log_dir)
test_writer = tf.summary.create_file_writer(test_log_dir)

In [29]:
def train():
    for epoch in range(10):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
        with train_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step = epoch)
            tf.summary.scalar('acc', train_accuracy.result(), step = epoch)
        print('Epoch{} loss is {}, accuracy is {}'.format(epoch,
                                                         train_loss.result(),
                                                         train_accuracy.result()))
        
        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(model, images, labels)
        with test_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step = epoch)
            tf.summary.scalar('acc', test_accuracy.result(), step = epoch)
        print('Epoch{} test_loss is {}, test_accuracy is {}'.format(epoch,
                                                         test_loss.result(),
                                                         test_accuracy.result()))    
        
        
        train_loss.reset_states() #重置对象
        train_accuracy.reset_states()
        
        test_loss.reset_states() #重置对象
        test_accuracy.reset_states()

In [30]:
train()

Epoch0 loss is 0.8024818301200867, accuracy is 0.7539499998092651
Epoch0 test_loss is 0.35624372959136963, test_accuracy is 0.8888000249862671
Epoch1 loss is 0.33778735995292664, accuracy is 0.8934166431427002
Epoch1 test_loss is 0.27743351459503174, test_accuracy is 0.9136000275611877
Epoch2 loss is 0.2772001326084137, accuracy is 0.9131166934967041
Epoch2 test_loss is 0.2523631155490875, test_accuracy is 0.9207000136375427
Epoch3 loss is 0.2472366988658905, accuracy is 0.9207000136375427
Epoch3 test_loss is 0.2194492220878601, test_accuracy is 0.9322999715805054
Epoch4 loss is 0.22750599682331085, accuracy is 0.9272333383560181
Epoch4 test_loss is 0.2222587615251541, test_accuracy is 0.9294000267982483
Epoch5 loss is 0.2144065499305725, accuracy is 0.9310666918754578
Epoch5 test_loss is 0.2003791183233261, test_accuracy is 0.9379000067710876
Epoch6 loss is 0.20544786751270294, accuracy is 0.9344333410263062
Epoch6 test_loss is 0.1903553009033203, test_accuracy is 0.9419000148773193
E

In [31]:
%load_ext tensorboard
%matplotlib inline

In [32]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 3784), started 19:01:24 ago. (Use '!kill 3784' to kill it.)